In [ ]:
# import sys
# !{sys.executable} -m pip install pyslim

In [1]:
import msprime
import pyslim
import numpy as np

## Recap and down sample (`recapitation.py`)

In [ ]:
slim_tree_path = "test.trees"
slim_params_path = "maize_scale_40.param"
N = 50
out_path = "final.trees"

In [ ]:
with open(slim_params_path, "r") as paramF:
    lines = paramF.readlines()

In [ ]:
mu = float(lines[0].strip())
rho = float(lines[1].strip())
Ne_recap = int(lines[2].strip().split()[1])

In [ ]:
print(mu, rho, Ne_recap)

In [ ]:
ts = pyslim.load(slim_tree_path)

In [ ]:
ts_recap = ts.recapitate(recombination_rate=rho, Ne=Ne_recap)

In [ ]:
len(ts_recap.samples())

In [ ]:
sampN = np.random.choice(ts_recap.samples(), size=N, replace=False)

In [ ]:
sampN

In [ ]:
ts_samp = ts_recap.simplify(samples=sampN)

## Add neutral mutations, produce GT mtx, output inferred and true trees

In [ ]:
#!{sys.executable} -m pip install scikit-allel

In [2]:
import sys, os
sys.path.append(os.path.join(os.path.dirname("__file__"), "../..")) # directory of module fea_util

In [3]:
from feature_util import run_RELATE

In [ ]:
help(run_RELATE)

In [4]:
samp_t_path = "/Volumes/mo/maiz/SLiM_trial_swp/SLiM_trial_swp_265_samp.trees"
slim_params_path = "maize_scale_40.param"
scale = 40

In [5]:
with open(slim_params_path, "r") as paramF:
    lines = paramF.readlines()
scaled_mu = float(lines[0].strip())
scaled_rho = float(lines[1].strip())
scaled_N0 = int(lines[-1].strip().split()[1])

In [6]:
mu = scaled_mu/scale
rho_cMpMb = scaled_rho/scale*100*1e6 # 1cM = 1e-2 crossover
N0 = scaled_N0*40

In [7]:
print(scaled_mu, scaled_rho, scaled_N0)
print(mu, rho_cMpMb, N0)
# lines

1.2e-06 2.79e-07 2501435
3e-08 0.6975 100057400


In [8]:
ts_samp = pyslim.load(samp_t_path)

In [9]:
ts_mut = msprime.mutate(ts_samp, rate=scaled_mu, keep=True) # random_seed=958, 

In [ ]:
#ts_mut.dump(out_path) # True gene tree

In [10]:
ts_mut.num_trees

2379

In [ ]:
test = [46854.23, 46888.26, 47982.21, 47982.25, 47982.68, 49998.63, 49998.95, 49999.12, 49999.58, 50000.0, 50000.12, 50001.26, 50001.98, 51215.66]

In [ ]:
var_ppos_ls = []
prev_pos = 0
for site in test:
    site_pos = int(site)
    if site_pos <= prev_pos:
        if prev_pos == 49999:
            var_ppos_ls.append(-1) # flag indicating this site should be removed
            continue
        else:
            site_pos = prev_pos + 1
    
    var_ppos_ls.append(site_pos)
    prev_pos = site_pos

In [ ]:
for i in range(len(test)):
    print(test[i], var_ppos_ls[i])

In [11]:
def get_site_ppos(ts):
    var_ppos_ls = []
    prev_pos = 0
    for site in ts.sites():
        site_pos = int(site.position)
        if site_pos <= prev_pos:
            if prev_pos == 49999:
                var_ppos_ls.append(-1) # flag indicating this site should be removed
                continue
            else:
                site_pos = prev_pos + 1
        var_ppos_ls.append(site_pos)
        prev_pos = site_pos
    return np.array(var_ppos_ls)

In [ ]:
# def get_site_ppos(ts):
#     var_ppos_ls = []
#     prev_pos = 0
#     for site in ts.sites():
#         site_pos = int(site.position)
#         if site_pos <= prev_pos: site_pos = prev_pos +1
#         var_ppos_ls.append(site_pos)
#         prev_pos = site_pos
#     return var_ppos_ls

In [ ]:
pos_ls = []
fl_pos_ls = []
for site in ts_mut.sites():
    pos_ls.append(int(site.position))
    fl_pos_ls.append(site.position)

In [12]:
ppos_ls = get_site_ppos(ts_mut)

In [27]:
#ts_mut.max_root_time
for tree in ts_mut.trees():
    print(tree.total_branch_length, tree.time(tree.root))

100861.64647962668 18167.470325759514
100091.64647962668 18167.470325759514
88800.75665416355 12522.025413027939
88841.75665416355 12522.025413027939
88019.55917057063 12522.025413027939
87879.55917057063 12522.025413027939
87856.50205665568 12522.025413027939
89023.50205665568 12522.025413027939
87419.71233954166 12522.025413027939
83292.94084751049 12522.025413027939
81313.98339890715 12522.025413027939
79942.98339890715 12522.025413027939
79758.98339890715 12522.025413027939
80136.98339890715 12522.025413027939
81273.98339890715 12522.025413027939
81167.98339890715 12522.025413027939
81153.98339890715 12522.025413027939
82122.98339890715 12522.025413027939
95768.43225838311 19344.74984276591
96046.4322583831 19344.74984276591
98025.38970698642 19344.74984276591
97833.38970698642 19344.74984276591
97898.38970698642 19344.74984276591
97528.38970698642 19344.74984276591
97807.38970698642 19344.74984276591
98045.38970698642 19344.74984276591
97545.38970698642 19344.74984276591
98411.963

81839.87196190689 8188.342935638699
81854.87196190689 8188.342935638699
82871.93943285808 8188.342935638699
83263.93943285808 8188.342935638699
82368.93943285808 8188.342935638699
82388.67809106805 8188.342935638699
83759.67809106805 8188.342935638699
81106.31592293334 8188.342935638699
81458.31592293334 8188.342935638699
84111.67809106805 8188.342935638699
84472.67809106805 8188.342935638699
78544.21670892595 7831.837128053489
78117.21670892595 7831.837128053489
102896.84117012392 20221.649358652474
106778.92601850227 22162.691782841644
107116.92601850227 22162.691782841644
79168.22832409636 8188.342935638699
79618.22832409636 8188.342935638699
79162.22832409636 8188.342935638699
79398.22832409637 8188.342935638699
79042.22832409637 8188.342935638699
80437.22832409637 8188.342935638699
80564.22832409637 8188.342935638699
80826.22832409637 8188.342935638699
79851.22832409637 8188.342935638699
80814.75152946298 8188.342935638699
80660.75152946298 8188.342935638699
82770.84618455725 8188

89178.48010776067 8498.636476050911
89209.16441207025 8498.636476050911
88970.16441207025 8498.636476050911
89018.16441207027 8498.636476050911
88985.16441207027 8498.636476050911
89824.7119088286 8498.636476050911
89501.7119088286 8498.636476050911
91772.19871850219 8709.673892107729
84477.56224245127 8709.673892107729
90048.66691624266 8709.673892107729
84255.33579344918 8498.636476050911
84149.33579344918 8498.636476050911
81249.37139664478 8498.636476050911
84987.0200796034 8709.673892107729
85493.02007960339 8709.673892107729
85343.02007960339 8709.673892107729
84945.65828499693 8510.992994804488
82233.80076184799 8510.992994804488
83104.80076184799 8510.992994804488
83442.80076184799 8510.992994804488
84112.13798462007 8510.992994804488
85588.72402203246 8510.992994804488
85162.72402203246 8510.992994804488
107080.24310519456 19463.574277008753
106502.69560843625 19463.574277008753
105957.69560843625 19463.574277008753
104741.8831813671 19463.574277008753
106025.8831813671 19463.

105313.90545670297 14943.54195697484
113973.2133432555 19273.19590025111
108796.21740272178 19273.19590025111
111806.40767320097 19273.19590025111
112316.40767320097 19273.19590025111
111992.40767320097 19273.19590025111
110300.78282883135 19273.19590025111
109437.94830828904 19273.19590025111
108909.94830828904 19273.19590025111
108511.94830828904 19273.19590025111
108454.94830828904 19273.19590025111
110146.57315265863 19273.19590025111
109879.57315265863 19273.19590025111
153015.11137510574 40840.965011474655
152872.11137510574 40840.965011474655
222093.9975065849 75451.90807721425
112510.60496832358 20660.211808083583
109397.45693373494 20660.211808083583
110515.45693373494 20660.211808083583
109807.46531414957 20660.211808083583
114796.23584885639 20660.211808083583
112934.76350688988 20660.211808083583
112300.76350688988 20660.211808083583
91860.99666129293 11110.13755177903
91863.99666129293 11110.13755177903
110964.14517390204 20660.211808083583
110977.14517390204 20660.2118080

118765.25874277051 24255.123402612062
118530.25874277051 24255.123402612062
117442.71681973057 24255.123402612062
117432.71681973057 24255.123402612062
115425.92755821621 24255.123402612062
115314.92755821621 24255.123402612062
115304.92755821621 24255.123402612062
112374.36177788624 24255.123402612062
82636.44599086801 9386.165509102959
82649.44599086801 9386.165509102959
82621.44599086801 9386.165509102959
80254.5627864072 9386.165509102959
80797.5627864072 9386.165509102959
80433.5627864072 9386.165509102959
79291.60206282054 9386.165509102959
77292.77421472936 8386.751585057365
78589.77421472936 8386.751585057365
78403.77421472936 8386.751585057365
79271.5959275491 8386.751585057365
82195.30386912315 8386.751585057365
83010.30386912315 8386.751585057365
83111.30386912315 8386.751585057365
81617.30386912315 8386.751585057365
78220.60370687107 8386.751585057365
78465.60370687107 8386.751585057365
78902.60370687107 8386.751585057365
83908.30386912315 8386.751585057365
83789.5970737197

108309.00393151624 12278.060832702107
108759.66254126311 12278.060832702107
108391.66254126311 12278.060832702107
108419.66254126311 12278.060832702107
108517.66254126311 12278.060832702107
104867.35049180743 12278.060832702107
104863.35049180743 12278.060832702107
104175.0329189739 12278.060832702107
104226.0329189739 12278.060832702107
106014.0519998755 12278.060832702107
105993.0519998755 12278.060832702107
105986.0519998755 12278.060832702107
105336.78119297537 12278.060832702107
106613.9371669437 12278.060832702107
106113.9371669437 12278.060832702107
106100.9371669437 12278.060832702107
105505.9371669437 12278.060832702107
105405.9371669437 12278.060832702107
105513.9371669437 12278.060832702107
103449.16838587048 11245.676442165503
103770.16838587048 11245.676442165503
98553.67672503067 11245.676442165503
100323.37218630515 11245.676442165503
100682.37218630515 11245.676442165503
100466.37218630515 11245.676442165503
100536.37218630515 11245.676442165503
100715.37218630515 11245

87509.81885123142 9422.143721023234
86899.81885123142 9422.143721023234
86672.81885123144 9422.143721023234
86709.81885123144 9422.143721023234
86637.81885123144 9422.143721023234
86653.81885123144 9422.143721023234
86650.81885123144 9422.143721023234
84852.54877853824 9422.143721023234
84869.54877853824 9422.143721023234
84848.54877853824 9422.143721023234
87179.12736975666 9422.143721023234
90008.75385417737 9422.143721023234
89953.75385417737 9422.143721023234
90339.75385417737 9422.143721023234
90551.75385417737 9422.143721023234
90122.75385417737 9422.143721023234
89988.75385417737 9422.143721023234
89035.75385417737 9422.143721023234
86381.58823744289 9422.143721023234
83560.37222488689 9422.143721023234
83767.37222488689 9422.143721023234
87335.77374875531 9422.143721023234
87011.77374875531 9422.143721023234
86590.77374875531 9422.143721023234
86893.77374875531 9422.143721023234
86878.77374875531 9422.143721023234
86648.77374875531 9422.143721023234
86363.77374875531 9422.14372

125884.21338587685 26413.48625304993
127109.08666840958 26413.48625304993
127047.08666840958 26413.48625304993
122731.90605823784 26413.48625304993


In [13]:
ts_mut.num_mutations

11809

In [14]:
len(ppos_ls)

11809

In [ ]:
for i in range(len(pos_ls)): print(fl_pos_ls[i], pos_ls[i], ppos_ls[i])

In [15]:
GTM = ts_mut.genotype_matrix()

In [16]:
GTM.shape

(11809, 200)

In [17]:
mask = (ppos_ls != -1)

In [18]:
np.sum(mask)

11809

In [19]:
ppos_ls = ppos_ls[mask]

In [20]:
GTM = GTM[mask, :]

In [21]:
GTM.shape

(11809, 200)

In [ ]:
str(mu)

In [22]:
ts_inf, _ = run_RELATE(ppos_ls, GTM, str(2*N0), rho_cMpMb=rho_cMpMb, mut_rate=str(mu))

Milestone: Running RELATE pipeline, try_1
Milestone: RELATE pipeline success


In [23]:
ts_inf

In [25]:
ts_inf.num_mutations

11756

In [26]:
for tree in ts_inf.trees():
    print(tree.total_branch_length, tree.time(tree.root))

3768279.6014060974 589962.0
3019886.0236854553 352533.53125
2951023.1069869995 345749.3125
2875849.57170105 566509.3125
3123802.758377075 540835.1875
3963166.7257003784 744482.25
4218212.770248413 977914.125
3603565.466781616 703985.0625
3004208.8597488403 510307.03125
2651800.984802246 331290.15625
2857483.089576721 274062.65625
2843847.679046631 354189.875
3343738.5154800415 454422.5625
3147500.100921631 381592.09375
3067181.6228637695 296891.125
3085211.2780838013 322424.53125
3366014.041366577 384256.1875
2947138.2337989807 338329.8125
2803220.5294570923 286828.75
3490880.7189483643 455724.71875
3630300.9603271484 415254.9375
3853915.7316589355 499121.5625
3748763.346216202 481986.84375
4008737.4035053253 578094.875
4018284.7600860596 742669.375
3929213.7520980835 667413.5625
3546309.0144081116 453238.5
4056414.4390506744 913331.875
3065315.4991226196 346121.125
3577870.8984680176 542780.0
3163940.786754608 376002.40625
3574333.470603943 646676.0
3766530.82976532 686549.5625
413557

2931443.21269989 301166.25
3886491.023994446 712008.25
3393225.0187339783 458059.09375
3685982.422027588 590687.1875
3628177.683860779 597109.375
3273682.9355392456 506812.71875
3227389.0096149445 494650.71875
3333247.855995178 457802.53125
3111647.491470337 501311.90625
3597068.2132530212 611319.9375
3691814.5410046577 655788.875
3257771.5494823456 514075.65625
2908496.4315013885 251526.484375
3455212.1345214844 481419.75
4078096.1355514526 833724.75
3470062.9669981003 497252.84375
3780703.803123474 483204.03125
3648059.136779785 547700.4375
4068085.1209335327 668432.875
3727774.1077804565 578333.75
2565229.893058777 214928.71875
3245471.5210113525 306435.59375
4274211.333709717 842723.0
3401263.73197937 434012.78125
4161646.479364872 717454.5625
5482906.8012161255 1219099.125
4791461.5113220215 1044607.4375
3100426.0908470154 342871.25
3554570.076980591 498886.125
3288647.0688705444 351977.5
2995118.754573822 250815.84375
3983043.976764679 815878.4375
4613454.350322723 851146.75
3806